### Import the required runtime library. The requests library is used to send requests for extracting data. The pandas library is used for the subsequent processing of the data.

In [ ]:
import requests
import pandas as pd
from pandas import json_normalize

### The api key I obtained after registering my account.

In [ ]:
apiKey = '321aae4fc32db61bc81ffaa61ef9b374'

### Get the data of a movie with number 500 using the method provided by the website.

In [ ]:
data_test = requests.get("https://api.themoviedb.org/3/movie/500?api_key=" + apiKey + "&language=en-US").json()
data_test

{'adult': False,
 'backdrop_path': '/jqFjgNnxpXIXWuPsyfqmcLXRo9p.jpg',
 'belongs_to_collection': None,
 'budget': 1200000,
 'genres': [{'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}],
 'homepage': '',
 'id': 500,
 'imdb_id': 'tt0105236',
 'original_language': 'en',
 'original_title': 'Reservoir Dogs',
 'overview': 'A botched robbery indicates a police informant, and the pressure mounts in the aftermath at a warehouse. Crime begets violence as the survivors -- veteran Mr. White, newcomer Mr. Orange, psychopathic parolee Mr. Blonde, bickering weasel Mr. Pink and Nice Guy Eddie -- unravel.',
 'popularity': 36.857,
 'poster_path': '/xi8Iu6qyTfyZVDVy60raIOYJJmk.jpg',
 'production_companies': [{'id': 285,
   'logo_path': None,
   'name': 'Live Entertainment',
   'origin_country': ''},
  {'id': 26198,
   'logo_path': None,
   'name': 'Dog Eat Dog Productions',
   'origin_country': ''}],
 'production_countries': [{'iso_3166_1': 'US',
   'name': 'United States of America'}],
 'rele

### Obviously, the above test is only for one movie, and I need to get more movies by different page numbers.

In [ ]:
def retrieveMovies(pageNumber):
    initial_url = 'https://api.themoviedb.org/3/movie/top_rated'
    two_para = {'api_key': apiKey, 'page': pageNumber}
    retrieve_Movies = requests.get(initial_url, params=two_para).json()
    return retrieve_Movies

### Extract movies information from 500 pages.

In [ ]:
movie_info = []
for page in range(1, 500):
    movie_info.append(retrieveMovies(page)['results']) #Results are followed by the specific information of the movie.

### Get movie id.

In [ ]:
def getMovieId():
    movieId = []
    for j in movie_info:
        for i in j:
            movieId.append(i['id'])
    return movieId

In [ ]:
movie_id = getMovieId()

### Extract the specific information of each movie by movie id.

In [ ]:
df = pd.DataFrame()

for id in movie_id:
    query = "https://api.themoviedb.org/3/movie/" + str(id) + "?api_key=" + apiKey + "&language=en-US"
    response = requests.get(query)
    if response.status_code==200: #The number 200 means the request response was successful.
        results = response.json()
        movies = json_normalize(results) #Used to parse json data to get dataframe.
        df = pd.concat([df, movies]) #Splice together with the dataframe created at the beginning.
df.head(5)

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,belongs_to_collection
0,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,An offer you can't refuse.,The Godfather,False,8.712,17641,230.0,The Godfather Collection,/9Baumh5J9N1nJUYzNkm0xsgjpwY.jpg,/3WZTxpgscsmoUk81TuECXdFOD0R.jpg,NaN
0,False,/wPU78OPN4BYEgWYdXyg0phMee64.jpg,25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,...,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.701,23480,NaN,NaN,NaN,NaN,NaN
0,False,/ejniJnlOdtSgtbh8D7u2RxT6Uli.jpg,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",https://videocine.com.mx/peliculas/cuando-sea-...,772071,tt12076762,es,Cuando Sea Joven,70-year-old Malena gets a second chance at lif...,...,,Cuando Sea Joven,False,8.642,212,NaN,NaN,NaN,NaN,NaN
0,False,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,13000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",,240,tt0071562,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,...,"I don't feel I have to wipe everybody out, Tom...",The Godfather Part II,False,8.597,10691,230.0,The Godfather Collection,/9Baumh5J9N1nJUYzNkm0xsgjpwY.jpg,/3WZTxpgscsmoUk81TuECXdFOD0R.jpg,NaN
0,False,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",http://www.schindlerslist.com/,424,tt0108052,en,Schindler's List,The true story of how businessman Oskar Schind...,...,"Whoever saves one life, saves the world entire.",Schindler's List,False,8.568,13886,NaN,NaN,NaN,NaN,NaN


### Retrieve the saved csv file.

In [ ]:
df.to_csv("movies_raw.csv",index=False)
df = pd.read_csv("movies_raw.csv")
df.head(5)

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,belongs_to_collection
0,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,An offer you can't refuse.,The Godfather,False,8.712,17641,230.0,The Godfather Collection,/9Baumh5J9N1nJUYzNkm0xsgjpwY.jpg,/3WZTxpgscsmoUk81TuECXdFOD0R.jpg,NaN
1,False,/wPU78OPN4BYEgWYdXyg0phMee64.jpg,25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,...,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.701,23480,NaN,NaN,NaN,NaN,NaN
2,False,/ejniJnlOdtSgtbh8D7u2RxT6Uli.jpg,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",https://videocine.com.mx/peliculas/cuando-sea-...,772071,tt12076762,es,Cuando Sea Joven,70-year-old Malena gets a second chance at lif...,...,NaN,Cuando Sea Joven,False,8.642,212,NaN,NaN,NaN,NaN,NaN
3,False,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,13000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,240,tt0071562,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,...,"I don't feel I have to wipe everybody out, Tom...",The Godfather Part II,False,8.597,10691,230.0,The Godfather Collection,/9Baumh5J9N1nJUYzNkm0xsgjpwY.jpg,/3WZTxpgscsmoUk81TuECXdFOD0R.jpg,NaN
4,False,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",http://www.schindlerslist.com/,424,tt0108052,en,Schindler's List,The true story of how businessman Oskar Schind...,...,"Whoever saves one life, saves the world entire.",Schindler's List,False,8.568,13886,NaN,NaN,NaN,NaN,NaN
